<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Naas Chat - Create plugin with commands

**Tags:** #naaschatplugin #naas #naas_driver #chat #plugin #ai

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel)

**Last update:** 2023-10-19 (Created: 2023-10-17)

**Description:** This notebook creates a Naas Chat plugin with commands.

**References:**
- [Naas Chat Documentation](https://site.naas.ai/docs/platform/aI-powered-chat)
- [Naas Chat Plugin driver](https://github.com/jupyter-naas/drivers/blob/main/naas_drivers/tools/naas_chat_plugin.py)

## Input

### Import libraries

In [5]:
import naas
from naas_drivers import naas_chat_plugin
import naas_data_product
from IPython.display import Markdown
from datetime import date, timedelta
import os
import pandas as pd

### Setup variables
**Mandatory**
- `input_dir`: Input directory to retrieve file from.
- `file_sales`: Name of the file to be retrieved.
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name`: Google Sheets sheet name.
- `name`: The name of the plugin.
- `system_prompt`: Plugin system prompt with the commands they can use.
- `model`: This variable holds the openai model.
- `temperature`: This variable holds the temperate value.
- `description`: This variable holds a string that describes the plugin.
- `avatar`: This variable holds a URL to an image to be displayed in your plugin as avatar.
- `input_dir`: This variable holds a relative path to the directory where input notebooks with commands will be stored.

**Optional**
- `output_dir`: This variable holds a relative path to the directory where output files will be stored. 
- `output_path`: This variable holds the full path of the plugin.

In [6]:
# Mandatory
input_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "sales-engine", date.today().isoformat())
file_sales = "sales"
spreadsheet_url = naas.secret.get("ABI_SPREADSHEET") or "YOUR_GOOGLE_SPREADSHEET_URL"
sheet_name = "SALES"
name = "⚡️ Sales Assitant"
model = "gpt-4"
temperature = 0
description = "This plugin is used to setup the Sales Engine"
avatar = ""
model_dir = os.path.join(naas_data_product.ROOT_PATH, "models", "sales-engine")
input_image = "sales_trend.png"
input_html = "sales_trend.html"

# Optional
output_dir = os.path.join(naas_data_product.ROOT_PATH, "outputs", "plugins")
os.makedirs(output_dir, exist_ok=True) # Create dirs
output_path = os.path.join(output_dir, f"{name.lower().replace(' ', '_')}.json")

## Model

### Get assets

In [7]:
image_url = get_image_asset(input_dir, input_image)
print("Image URL:", image_url)

image_html = get_image_asset(input_dir, input_html)
print("Image HTML:", image_html)

Image URL: 
Image HTML: 


### Get data from Google Sheets spreadsheet

In [11]:
def get_data(
    input_dir,
    input_file,
    spreadsheet_url,
    sheet_name
):
    # Init
    df = pd.DataFrame()
    
    # Load data from pickle
    data = pload(input_dir, input_file)    
    if data is None:
        data = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name)
    if isinstance(data, pd.DataFrame):
        df = data.copy()
        
    # Filter on this week and last week
    if len(df) > 0:
        tw = date.today().strftime("W%W-%Y")
        lw = (date.today() - timedelta(days=date.today().weekday() + 7)).strftime("W%W-%Y")
        to_drop = [
            "ENTITY",
            "SCENARIO",
            "STAFF_COUNT",
            "INTERACTION_SCORE",
            "MESSAGING_OPTIONS",
        ]
        df = df[(df["SCENARIO"].isin([tw, lw]))].drop(to_drop, axis=1)
    return df.reset_index(drop=True)

data = get_data(input_dir, file_sales, spreadsheet_url, sheet_name)
print("Data:", len(data))
data.head(len(data))

Data: 19


,DATE,FULLNAME,OCCUPATION,ICP,COMPANY_NAME,INDUSTRY,COUNTRY,STAFF_RANGE_NAME,INTERACTIONS,PROFILE_URL
0,Sun. 17 Dec.,Wes Lorenzini,Engineering User Happiness from Ground to Clou...,Developer,WL Technology,Software Development,US,Micro Team,Comment '=true' on 'Have you ever experienced ...,https://www.linkedin.com/in/ACoAAAGyHq4Bmiv1-d...
1,Sun. 17 Dec.,Mark Conway,Full Stack Data Scientist | Quant | Machine Le...,Developer,Ahold Delhaize,Retail,NL,Mega Corporation,Sent 'interest' reaction to 'Have you ever exp...,https://www.linkedin.com/in/ACoAAA-046sB3NR_hW...
2,Sun. 17 Dec.,David Mertz,"Consultant, Trainer, Data Scientist, Senior De...",Developer,KDM,Education Administration Programs,IL,NA,Sent 'praise' reaction to 'Good old days stori...,https://www.linkedin.com/in/ACoAAABXtC4B5bxAej...
3,Sun. 17 Dec.,Neil S.,Leadership | Analytics | Innovation | Commercial,Executive,nbn® Australia,Telecommunications,AU,Major Corporation,Sent 'empathy' reaction to 'Have you ever expe...,https://www.linkedin.com/in/ACoAAAsKemEB-ys9gp...
4,Sun. 17 Dec.,Sam Lausten,"Software Engineer, Machine Learning @ Torc Rob...",Developer,Torc Robotics,Truck Transportation,US,Enterprise Level,Comment 'I love seeing a problem in everyday l...,https://www.linkedin.com/in/ACoAACJY-hcBJ0lXu5...
5,Sun. 17 Dec.,Alvaro Franco Martins,Physicist | Data Scientist | Researcher | PhD ...,Developer,"State University of Maringa, Block C34",Education Administration Programs,BR,NA,Sent 'like' reaction to 'Have you ever experie...,https://www.linkedin.com/in/ACoAAC2UN3UBhU6Yz2...
6,Sun. 17 Dec.,Temidayo Omoniyi,Azure Data Engineer • Power BI Developer • Dat...,Developer,MSSQLTips.com,Software Development,US,Medium Company,Sent 'like' reaction to 'Have you ever experie...,https://www.linkedin.com/in/ACoAADA4TfoBv8iZNd...
7,Sun. 17 Dec.,Mokhtar Bendib,Tech Lead,Developer,Product Margin,Business Consulting and Services,US,Solopreneur,Sent 'empathy' reaction to 'Have you ever expe...,https://www.linkedin.com/in/ACoAACcqxvkBp_USVi...
8,Sun. 17 Dec.,Sathish Visanagiri,"Entrepreneur, Technologist & Teacher",Executive,Reckonsys Tech Labs,IT Services and IT Consulting,IN,Medium Company,Sent 'like' reaction to 'Have you ever experie...,https://www.linkedin.com/in/ACoAAADucJEBpx_Uad...
9,Sun. 17 Dec.,Johnnatan Lopez,"Payroll, Finance, Charisma, Python | I help yo...",Developer,EY,IT Services and IT Consulting,GB,Mega Corporation,Sent 'empathy' reaction to 'Have you ever expe...,https://www.linkedin.com/in/ACoAABrbZNsBb1fT8U...


### Create system prompt

In [12]:
system_prompt = f"""
Act as a Sales Assistant who has access to the Deals data from the CRM. 
Your role is to manage and nurture these deals, ensuring that they progress through the sales pipeline effectively. 
You will need to communicate with potential clients, understand their needs, and propose solutions that align with their goals. 
Utilize the information in the CRM to personalize your approach, track interactions, and manage follow-ups. 
Your ultimate goal is to close deals successfully, contributing to the overall sales targets of the company.
Start by presenting yourself.
The first message should be about presenting yourself with a maximum of 3 bullet points and displaying an image inside the markdown of the chat: ![{image_url}]({image_html}).
You must ALWAYS show the image in the first message.
Then, wait for the first answer from the user, and then start with the first high-level analysis. 
Here is the data from the content analytics that you should focus on: {data.to_string()}
"""

### Create commands

In [13]:
commands = create_command(model_dir)

👌 Well done! Your Notebook has been sent to production.



<IPython.core.display.Javascript object>

Button(button_style='primary', description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Notebook as API" feature, just replace .add by .delete
👌 Well done! Your Notebook has been sent to production.



...

PS: to remove the "Notebook as API" feature, just replace .add by .delete
👌 Well done! Your Notebook has been sent to production.



<IPython.core.display.Javascript object>

Button(button_style='primary', description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Notebook as API" feature, just replace .add by .delete
[
    {
        "name": "HubSpot_Connect",
        "avatar": "https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/HubSpot.png",
        "action": {
            "request_type": "POST",
            "url": "https://public.naas.ai/YWJpLTJEb3BlbnRlYW1zLTQwbmFhcy0yRWFp/notebook/5293c72ca775534607ed3c0f3271cd4d6d7290997d3dc104fdbd881de0cc",
            "payload": {
                "hs_access_token": {
                    "type": "str",
                    "description": "This variable stores an access token used for accessing the HubSpot API. This value will be stored under the secret 'HS_ACCESS_TOKEN'.",
                    "default": "None"
                }
            }
        },
        "description": "This command integrates your HubSpot access token into the Naas Lab secret manager."
    },
    {
        "name": "LinkedIn_Connect",
        "avatar": "https://raw.githubuse

## Output

### Create Naas Chat plugin
This function will generate the plugin in JSON format and also verify if your prompt adheres to the recommended limit, which is set at 20% of the maximum tokens allowed by the model. Then, it will save your plugin in your local environment.

In [15]:
# Create plugin
plugin_file_path = naas_chat_plugin.create_plugin(
    name=name,
    prompt=system_prompt,
    model=model,
    temperature=temperature,
    output_path=output_path,
    description=description,
    avatar=avatar,
    commands=commands
)

⚠️ Be careful, your system prompt looks too big.
                Tokens: 3181 (limit recommended: 20% -> 1638)
💾 Plugin successfully saved. You can use it in your Naas Chat with: /home/ftp/abi/outputs/plugins/⚡️_sales_assitant.json


### Create asset
This asset can be utilized by using the command `/use` in your Naas Chat or by simply clicking on the link provided in the last cell output.

In [16]:
plugin_url = naas.asset.add(plugin_file_path, params={"inline": True})
print(plugin_url)

👌 Well done! Your Assets has been sent to production.



<IPython.core.display.Javascript object>

Button(button_style='primary', description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Assets" feature, just replace .add by .delete
https://public.naas.ai/YWJpLTJEb3BlbnRlYW1zLTQwbmFhcy0yRWFp/asset/4221ab472867013c987a6bebd8da7a0c04cd82ebb281c3ddd57dc537cd9f


### Create new chat
NB: You don't need to click on 'Create New Chat' everytime you update your system prompt, you can use the command `/refresh`.

In [17]:
Markdown(f"[Create New Chat](https://naas.ai/chat/use?plugin_url={plugin_url})")

[Create New Chat](https://naas.ai/chat/use?plugin_url=https://public.naas.ai/YWJpLTJEb3BlbnRlYW1zLTQwbmFhcy0yRWFp/asset/4221ab472867013c987a6bebd8da7a0c04cd82ebb281c3ddd57dc537cd9f)